In [4]:
# Librerías
import os
import json
import numpy as np
import math
import matplotlib.pyplot as plt
import xarray as xr
import folium
import geopandas as gpd
from datetime import datetime, timedelta
from shapely.geometry import shape, Polygon
from dotenv import load_dotenv

# Sentinel Hub specific imports
from sentinelhub import (
    CRS,
    BBox,
    BBoxSplitter,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    SentinelHubCatalog,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    read_data,
    SHConfig
)

# Rasterio for raster operations
import rasterio as rio
from rasterio import features
from rasterio.merge import merge
from rasterio.plot import show

# Proj4 for coordinate transformations (if needed, though geopandas handles this well)
from pyproj import Transformer

In [5]:
# Configuración de Sentinel Hub
load_dotenv()
config = SHConfig()
config.sh_client_id = os.getenv("SH_CLIENT_ID")
config.sh_client_secret = os.getenv("SH_CLIENT_SECRET")
catalog = SentinelHubCatalog(config=config)

In [7]:
# Configuración de parametros
aoi_geojson_path = "aoi.geojson"  
start_date = "2025-01-01"
end_date = "2025-01-31"
resolution = 30   # metros
burn_threshold = 0.15
output_folder = "results"
os.makedirs(output_folder, exist_ok=True)

In [8]:
#Cargar AOI
with open(aoi_geojson_path, "r", encoding="utf-8") as f:
    geojson_aoi_data = json.load(f)

aoi = shape(geojson_aoi_data["features"][0]["geometry"])
aoi_bbox = BBox(bbox=aoi.bounds, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

In [9]:
#Dividir AOI en Tiles
def calculateGridSize(bbox_size):
    width, height = bbox_size
    tiles_x = math.ceil(width / 1200)
    tiles_y = math.ceil(height / 1200)
    return (tiles_x, tiles_y)

grid_size = calculateGridSize(aoi_size)
bbox_splitter = BBoxSplitter([aoi], CRS.WGS84, grid_size, reduce_bbox_sizes=True)

In [10]:
#Eval del indice de quemas
def get_evalscript_burn(threshold):
    return f"""
    //VERSION=3
    function setup() {{
      return {{
        input: ["B08", "B12"],
        output: {{
          bands: 1,
          sampleType: "FLOAT32"
        }}
      }};
    }}

    function evaluatePixel(sample) {{
      let nbr = (sample.B08 - sample.B12) / (sample.B08 + sample.B12);
      return [nbr > {threshold} ? 1 : 0];
    }}
    """

In [11]:
#Crear request por tile
def get_request_by_subarea(bbox, evalscript, start_date, end_date):
    size = bbox_to_dimensions(bbox, resolution=resolution)
    return SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(start_date, end_date),
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        data_folder=output_folder,
        config=config,
    )

evalscript_burn = get_evalscript_burn(burn_threshold)
bbox_list = bbox_splitter.get_bbox_list()
sh_requests_burn = [
    get_request_by_subarea(bbox, evalscript_burn, start_date, end_date)
    for bbox in bbox_list
]


In [12]:
#Descargar datos
dl_requests = [request.download_list[0] for request in sh_requests_burn]
downloaded_data = SentinelHubDownloadClient(config=config).download(dl_requests, max_threads=5)

# Guardar TIFFs descargados
tiffiles = []
for folder, _, files in os.walk(output_folder):
    for file in files:
        if file.lower().endswith((".tif",".tiff")):
            tiffiles.append(os.path.join(folder, file))

print(f"Descargados {len(tiffiles)} tiles")

Descargados 1 tiles


In [13]:
#Convertir a poligonos
polygons = []
for tif in tiffiles:
    with rio.open(tif) as src:
        raster_data = src.read(1)  # primera banda
        transform = src.transform
        crs = src.crs

        mask = raster_data == 1  # solo quemado
        for geom, value in features.shapes(raster_data.astype(np.int16),
                                           mask=mask,
                                           transform=transform):
            if value == 1:
                polygons.append({"geometry": shape(geom), "state": "Quemado"})
                
# Convertir a GeoDataFrame
gdf = gpd.GeoDataFrame(polygons, crs="EPSG:4326")

# Disolver polígonos por estado
if not gdf.empty:
    dissolved = gdf.dissolve(by="state", as_index=False)
    out_fp = os.path.join(output_folder, "burn_areas.geojson")
    dissolved.to_file(out_fp, driver="GeoJSON")
    print(f"✅ GeoJSON generado: {out_fp}")
else:
    print("⚠️ No se detectaron áreas quemadas en este rango de fechas")
    dissolved = None

✅ GeoJSON generado: results\burn_areas.geojson
